## Working with Pandas

In this section, we'll be using data from the UNHCR (UN Refugee Agency). The UNHCR has collected and published data on refugees, asylum seekers, and other "populations of concern" from 1951-2016; more information can be found at http://popstats.unhcr.org/en/overview.

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
# Reading in the first table

persons = pd.read_csv('UN_refugee_data/unhcr_popstats_export_persons_of_concern_all_data.csv')

Let's look at the data frame to see what's going on.

In [ ]:
# Showing top 5 rows

persons.head()

In [ ]:
# Showing last 5 rows

persons.tail()

In [ ]:
# Let's get summary information

persons.info()

So we see two issues - one is that the header hasn't been assigned properly, and the other is that we have asterisks which we'd like to replace with NaN's (null values).

In [ ]:
# Re-read in the data

persons = pd.read_csv('UN_refugee_data/unhcr_popstats_export_persons_of_concern_all_data.csv', header=3, na_values = '*')

In [ ]:
# Checking if we resolved the issues

persons.tail()

In [ ]:
persons.info()

There are a lot of parameters you can adjust when reading in a CSV file; see the documentation for details: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

Some of the column names are long - let's replace them for convenience of analysis.

In [ ]:
per_renamed = persons.rename(index=str, columns ={'Country / territory of asylum/residence': 'Residence',
                                    'Refugees (incl. refugee-like situations)': 'Refugees',
                                    'Asylum-seekers (pending cases)': 'Asylum-seekers',
                                    'Internally displaced persons (IDPs)': 'IDPs'})
per_renamed.head()

How do we look at specific columns?

In [ ]:
per_renamed['Origin']

How many countries have refugees and asylum-seekers come from?

In [ ]:
# Count unique values in "Residence" column (we'll count "Various/Unknown" as one country)

per_renamed['Origin'].nunique()

Let's say we want to focus on persons from Somalia.

In [ ]:
somali = per_renamed[per_renamed['Origin'] == 'Somalia']
somali

Now, let's say we want to be more specific and focus on Somalis who have come to the U.S. between 2000-2016.

In [ ]:
somali_us = somali[(somali['Residence'] == 'United States of America') & (somali['Year'] >= 2000) & (somali['Year'] <= 2016)]
somali_us

We can create a plot to look at the numeric data over time.

In [ ]:
# Selecting the population-data columns that do not have null values

pops = somali_us.select_dtypes(include=['float'])
pop_nonnull = pops.dropna(axis=1, how='any')
pop_nonnull.columns

In [ ]:
# Plotting the time series

pl = somali_us.plot(x='Year', y=pop_nonnull.columns)
pl.set_ylabel('Persons')
pl.set_title('Somali-to-U.S.')

Now, let's go back to the larger data frame, per_renamed.

In [ ]:
per_renamed.head()

You may have noticed the index on the leftmost side of the data frame. If we don't assign the index when we read in the data, pandas will automatically assign the row number as the index. But for faster and more convenient look-ups, you will often want to assign one of your columns as the index.

In [ ]:
# Let's set "Origin" as our new index and sort the countries alphabetically

origin_df = per_renamed.set_index('Origin').sort_index()
origin_df

This is convenient, because now we can more quickly access certain rows and columns, like so:

In [ ]:
# Choosing rows corresponding to the slice between Somalia and Sudan (inclusive), and the first four columns

origin_df.loc['Somalia':'Sudan', 'Year':'Asylum-seekers']

If you wanted to locate data by row/column numbers, you could use .iloc (integer-based location) instead of .loc (label-based location).

In [ ]:
origin_df.iloc[0, 0:4]

Now, as we've alluded to previously, there are a lot of null values in this table.

In [ ]:
# Quick way to check if there are any null values in each column

pd.isnull(origin_df).any()

In [ ]:
# Are there columns with ONLY null values?

pd.isnull(origin_df).all()

In [ ]:
# Which rows have null values?

origin_df[pd.isnull(origin_df).any(axis=1)].head(10)    # only want to look at the top 10 rows to save space

In [ ]:
# Which rows do NOT have null values?

origin_df[pd.notnull(origin_df).all(axis=1)].head(10)

There are several ways you might want to handle missing values, depending on your data and application. One way is to simply drop rows or columns with null values:

In [ ]:
# Dropping rows with NaN's in "Refugees" or "IDPs" columns

origin_df_nonnull = origin_df.dropna(axis=0, how='any', subset=['Refugees', 'IDPs'])
origin_df_nonnull.head(10)

However, sometimes dropping columns isn't feasible or desirable. You may instead want to impute missing values, replacing them with values that make sense in the context. 

Let's say we want to replace each NaN with the average value in its respective column.

In [ ]:
%%time
# First, calculate the mean for each column (this will take ~ 15-20 seconds)

means = origin_df.mean(axis=0)

In [ ]:
means

In [ ]:
# Now, replace the NaN's

origin_df.fillna(means).head(10)

Alternatively, you might want to replace null values with zeros.

In [ ]:
origin_df.fillna(0).head(10)

Let's look at some operations we can do in pandas.

In [ ]:
# Adding a new column representing the sum of "Refugees" and "Returned refugees" to per_renamed, the non-index-specified
# dataframe (treating NaNs as zeros for the calculation)

per_renamed['All refugees'] = per_renamed['Refugees'] + per_renamed['Returned refugees'].fillna(0)
per_renamed

What if we want to look at the total number of refugees (not Returned) from each country in "Origin" by year? The easiest way to do this is by grouping the data.

In [ ]:
# Grouping data by Origin and Year, then calculating totals

grouped_total = per_renamed.groupby(['Origin','Year'])['Refugees'].sum()
grouped_total

What if we want to find out which country "produced" (for lack of a better word) the most refugees, by year?

In [ ]:
# First, group and sum over each group

grouped_new = per_renamed.groupby(['Year', 'Origin'])['Refugees'].sum()
grouped_new

In [ ]:
# We can use the DataFrame.unstack() method to pivot the years to columns

unstacked = grouped_new.unstack(level=0)
unstacked

In [ ]:
# Finally, use the DataFrame.idxmax() method to return the indices corresponding to the maximum value for each column

max_origin = unstacked.idxmax(axis=0)
max_origin

To visualize this, let's use the visualization library Seaborn.

In [ ]:
import seaborn as sns

In [ ]:
# Resetting "Year" to be a column for easier plotting, and renaming the country column label from the assigned "0" to Country

max_origin_new = max_origin.reset_index()
max_origin_df = max_origin_new.rename(index=str, columns={0: 'Country'})
max_origin_df.head()

In [ ]:
# Swarm plot showing which country produced the most refugees by year

sns.swarmplot(x='Year', y='Country', data=max_origin_df)

The last thing we'll look at is merging two tables together. Another table provided by the UNHCR is a table providing the number of resettlement arrivals for each Residence/Origin country pair between 1959-2016.

In [ ]:
# Reading in resettlement table
 
resettle = pd.read_csv('UN_refugee_data/unhcr_popstats_export_resettlement_all_data.csv', header=3, na_values = '*')
resettle.head()

How can we merge this data with the persons-of-concern data?

First, let's rename the Residence column to match the naming convention in the per_renamed data frame.

In [ ]:
# Also, rename "Value" to "Resettled"

resettle_renamed = resettle.rename(index=str, columns ={'Country / territory of asylum/residence': 'Residence',
                                                        'Value': 'Resettled'})
resettle_renamed.head()

In [ ]:
# Remember per_renamed?

per_renamed.head()

It would make sense to merge on the combination of Year, Residence, and Origin. Also, since the resettlement data starts in 1959, we need to think about whether we want to keep the rows in per_renamed corresponding to the years between 1951 and 1959.

In [ ]:
# Let's say we want to keep the earlier data

left_join = pd.merge(per_renamed, resettle_renamed, how='left', on = ['Year', 'Residence', 'Origin'])
left_join[left_join['Residence'] == 'Canada']    # focusing on those who resettled in Canada

In [ ]:
# What if we only want to see the intersections of the two tables (where Year, Residence, and Origin all match up?)

inner_join = pd.merge(per_renamed, resettle_renamed, how='inner', on = ['Year', 'Residence', 'Origin'])
inner_join.head(10)

**Exercise 2:**

A third table from UNHCR is a demographics table showing the gender and age breakdown on "persons of concern" in each region of a given residence country. Focusing on adults between the ages of 18-59 in the year 2016, determine the proportion of persons who are female in each country of residence, and find the countries which took in less than 25% female persons of concern in that year.

In [ ]:
# Reading in demographics data

demographics =  pd.read_csv('UN_refugee_data/unhcr_popstats_export_demographics_all_data.csv', header=3, na_values = '*')

**Answer 2:**

In [ ]:
## ENTER CODE HERE


There are so many things you can do with pandas - we've just scratched the surface. The best way to learn it is really to get your hands dirty and start playing around with data. You may also want to check out the tutorials listed under *References*.

*References*:

The following materials were consulted during development of this notebook (the concept of calculating the proportion of female persons-of-concern in Assignment 2 was taken directly from Brandon Rhodes's tutorial):

John Gosset, April Wright (eds): "Data Carpentry Python Ecology lesson." Version 2017.04.0, April 2017, http://www.datacarpentry.org/python-ecology-lesson/.

Brandon Rhodes, PyCon 2015 Pandas Tutorial Materials, 2015, GitHub repository, https://github.com/brandon-rhodes/pycon-pandas-tutorial.

*Data:*

All refugee data used in this notebook comes from the UNHCR and can be found at http://popstats.unhcr.org under the Persons of Concern, Resettlement, and Demographics tabs, respectively. Data was exported following selection of appropriate checkboxes corresponding to the maximum amount of data output (e.g. "All Years" under the "Years" drop-down menu).